## Import parameters and libraries

In [1]:
import torch
import torch.nn as nn
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
# from google.colab import drive

# drive.mount("/content/drive")

# TRAIN_IMG_PATH = "/content/drive/MyDrive/AI/Cityscape/images_prepped_train-20230811T065243Z-001/images_prepped_train"
# TRAIN_MASK_PATH = "/content/drive/MyDrive/AI/Cityscape/annotations_prepped_train-20230811T065240Z-001/annotations_prepped_train"
# VAL_IMG_PATH = "/content/drive/MyDrive/AI/Cityscape/images_prepped_test-20230811T065241Z-001"
# VAL_MASK_PATH = "/content/drive/MyDrive/AI/Cityscape/annotations_prepped_test-20230811T065240Z-001"

# EDA_DS_PATH = "/content/drive/MyDrive/AI/Advance CV - Project 3/Dataset"
# EDA_TRAIN_LABELS = os.path.join(EDA_DS_PATH, "train_labels.csv")
# EDA_TRAIN_FILES = os.path.join(EDA_DS_PATH, "train_files.csv")
# EDA_VAL_FILES = os.path.join(EDA_DS_PATH, "val_files.csv")

# MODEL_PATH = "/content/drive/MyDrive/AI/Advance CV - Project 3/models/FCN8s"

In [3]:
TRAIN_IMG_PATH = "./dataset/bootcamp/train/images"
TRAIN_MASK_PATH = "./dataset/bootcamp/train/annotations"
VAL_IMG_PATH = "./dataset/bootcamp/test/images"
VAL_MASK_PATH = "./dataset/bootcamp/test/annotations"

EDA_DS_PATH = "./dataset/bootcamp/train"
EDA_TRAIN_LABELS = os.path.join(EDA_DS_PATH, "train_labels.csv")
EDA_TRAIN_FILES = os.path.join(EDA_DS_PATH, "train_files.csv")
EDA_VAL_FILES = os.path.join(EDA_DS_PATH, "val_files.csv")

MODEL_PATH = "./models/UNet"
METRIC_PATH = "./metrics"

os.makedirs(MODEL_PATH, exist_ok=True)  # Ensure directory exist

In [4]:
files_df = pd.read_csv(EDA_TRAIN_FILES)
labels_df = pd.read_csv(EDA_TRAIN_LABELS)
val_files_df = pd.read_csv(EDA_VAL_FILES)

In [5]:
print(files_df.head())
print()
print(labels_df.head())
print()
train_files = files_df['filename'].to_numpy()
print(train_files[:5])
print()
val_files = val_files_df['filename'].to_numpy()
print(val_files[:5])

            filename
0  0001TP_006690.png
1  0001TP_006720.png
2  0001TP_006750.png
3  0001TP_006780.png
4  0001TP_006810.png

   class     label
0      0       sky
1      1  building
2      2      pole
3      3      road
4      4  sidewalk

['0001TP_006690.png' '0001TP_006720.png' '0001TP_006750.png'
 '0001TP_006780.png' '0001TP_006810.png']

['0016E5_07959.png' '0016E5_07961.png' '0016E5_07963.png'
 '0016E5_07965.png' '0016E5_07967.png']


## Datasets Loader

In [6]:
# refer to dataset.py
from dataset import CityScapes

In [7]:
train_datasets = CityScapes(TRAIN_IMG_PATH, TRAIN_MASK_PATH, train_files)
image, mask = train_datasets.__getitem__(0)
print(image)
print()
print(mask)
print()
print(image.shape)
print()
print(mask.shape)

val_datasets = CityScapes(VAL_IMG_PATH, VAL_MASK_PATH, val_files)

tensor([[[-1.8439, -1.9124, -1.8782,  ..., -1.8268, -1.8610, -1.8268],
         [-1.8782, -1.8782, -1.8953,  ..., -1.8268, -1.8610, -1.8268],
         [-1.8953, -1.8610, -1.8610,  ..., -1.9124, -1.8782, -1.8439],
         ...,
         [-1.8610, -1.8610, -1.8439,  ..., -1.7240, -1.6727, -1.6727],
         [-1.8439, -1.8439, -1.8439,  ..., -1.6898, -1.6213, -1.7240],
         [-1.8439, -1.8610, -1.8439,  ..., -1.7240, -1.7240, -1.7412]],

        [[-1.7556, -1.8256, -1.7906,  ..., -1.6681, -1.7031, -1.7206],
         [-1.7906, -1.7906, -1.8081,  ..., -1.6681, -1.7031, -1.7206],
         [-1.8081, -1.7731, -1.7731,  ..., -1.7556, -1.7206, -1.7381],
         ...,
         [-1.7556, -1.7556, -1.7381,  ..., -1.4580, -1.4055, -1.3880],
         [-1.7381, -1.7381, -1.7381,  ..., -1.4580, -1.3704, -1.4405],
         [-1.7556, -1.7556, -1.7381,  ..., -1.4580, -1.4405, -1.4580]],

        [[-1.5256, -1.5953, -1.5604,  ..., -1.4210, -1.4559, -1.4384],
         [-1.5604, -1.5604, -1.5779,  ..., -1

In [8]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_datasets, batch_size=32, shuffle=True, num_workers=0)
val_dataloader = DataLoader(val_datasets, batch_size=32, shuffle=True, num_workers=0)

## Model Architecture

In [9]:
from architecture import UNET

## Engine function

- Dice coefficient metrics

In [10]:
from engine import DiceLoss
from engine import DiceCrossEntropyLoss
from engine import train

Z:\BEKA\2025\AI\Bootcamp\Source\Advanced CV\Project 3\engine.py:138: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  def train_engine(dataloader, model, loss_fn, optim, scaler=torch.cuda.amp.GradScaler()):


## Training UNET with DiceLoss

In [11]:
model = UNET(in_channels=3, out_channels=len(labels_df))
model.cuda()
loss_fn = DiceLoss()
optim = torch.optim.Adam(model.parameters(), lr = 1e-3)
print(model.out_channels)

12


In [12]:
model, logs = train(train_dataloader, val_dataloader, model, loss_fn, optim, 100, log_freq=1, save_best_model=True, 
              best_model_name='best_model_dice.pth', last_model_name='last_model_dice.pth', save_path=MODEL_PATH)
print(f"CUDA GPU: {torch.cuda.get_device_name(0)}")
print(f"Architecture: UNET DiceLoss")

100%|██████████████████████████████████████████████████████████████████████| 12/12 [02:10<00:00, 10.89s/it, loss=0.868]
Z:\BEKA\2025\AI\Bootcamp\Source\Advanced CV\Project 3\engine.py:184: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
100%|█████████████████████████████████████████████████████████████████████████| 4/4 [00:09<00:00,  2.27s/it, loss=0.93]


Epoch 0/99
----------
Train Loss: 0.8928, Val Loss: 0.9306
✅ Best model saved! (Val Loss: 0.9305686801671982)



100%|████████████████████████████████████████████████████████████████████████| 4/4 [00:09<00:00,  2.42s/it, loss=0.844]


Epoch 1/99
----------
Train Loss: 0.8496, Val Loss: 0.8440
✅ Best model saved! (Val Loss: 0.8440087586641312)



100%|█████████████████████████████████████████████████████████████████████████| 4/4 [00:08<00:00,  2.16s/it, loss=0.82]


...

Epoch 97/99
----------
Train Loss: 0.5638, Val Loss: 0.7066



100%|████████████████████████████████████████████████████████████████████████| 4/4 [00:08<00:00,  2.22s/it, loss=0.713]


Epoch 98/99
----------
Train Loss: 0.5632, Val Loss: 0.7126



100%|████████████████████████████████████████████████████████████████████████| 4/4 [00:08<00:00,  2.00s/it, loss=0.718]

Epoch 99/99
----------
Train Loss: 0.5624, Val Loss: 0.7120

Training complete in 249m 41s
Best test (smallest validation lost): 0.702685
CUDA GPU: NVIDIA GeForce RTX 4060 Laptop GPU
Architecture: UNET DiceLoss


In [13]:
logs_df = pd.DataFrame(logs)
logs_df.to_csv(os.path.join(METRIC_PATH, 'train_unet_dice.csv'), index=False)

## Training UNET with CrossEntropyLoss

In [14]:
model = UNET(in_channels=3, out_channels=len(labels_df))
model.cuda()
loss_fn = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr = 1e-3)
print(model.out_channels)

12


In [ ]:
model, logs = train(train_dataloader, val_dataloader, model, loss_fn, optim, 100, log_freq=1, save_best_model=True, 
              best_model_name='best_model_cross.pth', last_model_name='last_model_cross.pth', save_path=MODEL_PATH)
print(f"CUDA GPU: {torch.cuda.get_device_name(0)}")
print(f"Architecture: UNET CrossEntropyLoss")

100%|█████████████████████████████████████████████████████████████████████████| 4/4 [00:09<00:00,  2.29s/it, loss=1.99]


Epoch 0/99
----------
Train Loss: 1.6746, Val Loss: 2.0034
✅ Best model saved! (Val Loss: 2.003401517868042)



100%|█████████████████████████████████████████████████████████████████████████| 4/4 [00:09<00:00,  2.35s/it, loss=3.06]


Epoch 1/99
----------
Train Loss: 1.1927, Val Loss: 3.0781



100%|█████████████████████████████████████████████████████████████████████████| 4/4 [00:09<00:00,  2.30s/it, loss=1.36]


...

100%|████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.52it/s, loss=0.313]


Epoch 96/99
----------
Train Loss: 0.1328, Val Loss: 0.3128



100%|████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.61it/s, loss=0.382]


Epoch 97/99
----------
Train Loss: 0.1316, Val Loss: 0.3450



 25%|█████████████████▊                                                     | 3/12 [00:13<00:40,  4.47s/it, loss=0.124]

In [ ]:
logs_df = pd.DataFrame(logs)
logs_df.to_csv(os.path.join(METRIC_PATH, 'train_unet_cross.csv'), index=False)

## Training UNET with DiceLoss + CrossEntropyLoss

In [ ]:

model = UNET(in_channels=3, out_channels=len(labels_df))
model.cuda()
loss_fn = DiceCrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr = 1e-3)
print(model.out_channels)

In [ ]:
model, logs = train(train_dataloader, val_dataloader, model, loss_fn, optim, 100, log_freq=1, save_best_model=True, 
              best_model_name='best_model_combined.pth', last_model_name='last_model_combined.pth', save_path=MODEL_PATH)
print(f"CUDA GPU: {torch.cuda.get_device_name(0)}")
print(f"Architecture: UNET DiceLoss+CrossEntropyLoss")

In [ ]:
logs_df = pd.DataFrame(logs)
logs_df.to_csv(os.path.join(METRIC_PATH, 'train_unet_combined.csv'), index=False)